# Tutorial: Boosting

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import mne
import numpy as np
import sys; sys.path.insert(0, '../')
from esinet import util
from esinet.simulation import Simulation
import os

plot_params = dict(surface='white', hemi='both', verbose=0)

## Load the data
Just as in the [mne-python tutorial](https://mne.tools/stable/auto_tutorials/inverse/30_mne_dspm_loreta.html) we have to load some sample data first

In [2]:
data_path = mne.datasets.sample.data_path()
raw_fname = os.path.join(data_path, 'MEG', 'sample',
                    'sample_audvis_filt-0-40_raw.fif')

raw = mne.io.read_raw_fif(raw_fname, verbose=0)  # already has an average reference
events = mne.find_events(raw, stim_channel='STI 014', verbose=0)

event_id = dict(aud_l=1)  # event trigger and conditions
tmin = -0.2  # start of each epoch (200ms before the trigger)
tmax = 0.5  # end of each epoch (500ms after the trigger)
raw.info['bads'] = ['MEG 2443', 'EEG 053']
baseline = (None, 0)  # means from the first instant to t = 0
reject = dict(mag=4e-12, eog=150e-6)

epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=('mag', 'eog'), baseline=baseline, reject=reject,
                    verbose=0)

fname_fwd = data_path + '/MEG/sample/sample_audvis-meg-oct-6-fwd.fif'
fwd = mne.read_forward_solution(fname_fwd, verbose=0)

## Simulate Data
Artificial neural networks need training data to learn how to predict brain-electric activity (sources) given the M/EEG data. 

First, we calculate the signal to noise ratio (SNR) of our EEG data so we can adjust our simulations to it. 

In [3]:
target_snr = util.calc_snr_range(epochs, baseline_span=(-0.2, 0.0), data_span=(0.05, 0.2))
print(f'The target SNR ranges from {target_snr[0]:.2f} to {target_snr[1]:.2f}')

settings = dict(duration_of_trial=0, target_snr=target_snr, n_sources=(1, 10), extents=(2, 40), beta=1)
n_samples = 20000
simulation = Simulation(fwd, epochs.info, settings=settings, verbose=False)
simulation.simulate(n_samples=n_samples)


Removing projector <Projection | Average EEG reference, active : True, n_channels : 60>
The target SNR ranges from 0.88 to 14.34


  0%|          | 0/20000 [00:00<?, ?it/s]


Create EEG trials with noise...


  0%|          | 0/20000 [00:00<?, ?it/s]


Convert EEG matrices to a single instance of mne.Epochs...


c:\Users\lukas\Dokumente\projects\esinet\tutorials\..\esinet\util\util.py:99: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs.set_eeg_reference('average', projection=True, verbose=0)


# Boosting Model

In [4]:
from esinet.net import BoostNet

net = BoostNet(fwd, n_nets=5)
net.fit(simulation, epochs=50)

C:\Users\lukas\virtualenvs\esienv\lib\site-packages\scipy\optimize\_minimize.py:783: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "


new sample weights: mean=0.18856159752267931 +- 0.2604525533690105
new sample weights: mean=0.19322846154838685 +- 0.21400063909633293
new sample weights: mean=0.1859415150721003 +- 0.2254713599817257
new sample weights: mean=0.19056191336855563 +- 0.21633941054151423
new sample weights: mean=0.18986052297832418 +- 0.22646247487365942


## Evaluate the Boosting Model
First, we simulate fresh data:

In [5]:
n_samples = 200
simulation_test = Simulation(fwd, epochs.info, settings=settings, verbose=False)
simulation_test.simulate(n_samples=n_samples)

  0%|          | 0/200 [00:00<?, ?it/s]


Create EEG trials with noise...


  0%|          | 0/200 [00:00<?, ?it/s]


Convert EEG matrices to a single instance of mne.Epochs...


c:\Users\lukas\Dokumente\projects\esinet\tutorials\..\esinet\util\util.py:99: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  epochs.set_eeg_reference('average', projection=True, verbose=0)


### Then we evaluate the Boosting Model and its sub-nets visually

In [6]:
labels = ['Boosting Model']
mse = [net.evaluate_mse(simulation_test)]
for i, subnet in enumerate(net.nets):
    mse.append(subnet.evaluate_mse(simulation_test))
    labels.append('subnet ' + str(i+1))

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib qt
sns.boxplot(data=mse)

z = simulation_test.source_data.plot(**plot_params)
a = net.predict(simulation_test).plot(**plot_params)
b = net.nets[0].predict(simulation_test).plot(**plot_params)
c = net.nets[-1].predict(simulation_test).plot(**plot_params)

[print(np.mean(m)) for m in mse]


C:\Users\lukas\virtualenvs\esienv\lib\site-packages\scipy\optimize\_minimize.py:783: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  warn("Method 'bounded' does not support relative tolerance in x; "
